## Script to identify the Snow Cover (SC) status on the ground and calculate the Snow Cover Duration (SCD)

This script was used to clear all cloud- and no-data flagged pixels for each hydrological year and to interpolate these data/ cloud gaps to identify the Snow Cover status on the ground. Additionally, it was used to calculate the Snow Cover Duration for each pixels of the current hydrological year within the study region (Central Asia).

#### Before running this script: 

1. Use the script Download_MODIS.ipynb to download the needed MODIS Snow Cover Terra and Aqua data
2. Use the script ReplaceMissing_MODIS.ipynb to search for missing MODIS Terra files and replace them with the corresponding MODIS Aqua files

In [ ]:
# load the needed modules
import os
import shutil
import xarray
import h5py
import glob
import sys
import rasterio
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd

In [ ]:
# define needed folders, files and variables

# folders
# MODIS Terra folder: e.g. MOD10A1_20-21
MODIS_terra_file_folder = 'path/to/your/MODIS_Terra_folder/'
MODIS_aqua_file_folder = 'path/to/your/MODIS_Aqua_folder/'
# Snow cover folder: e.g. SC_20-21
SC_output_folder = 'path/to/your/SC_output_folder/'
SCD_output_folder = 'path/to/your/SCD_output_folder/'

# files
SRTM = 'path/to/h23v04_srtm_subset_corrected.tif'
SCD_output_array = 'SCD_current_hydrological_year.npz'
SCD_output_raster = 'SCD_current_hydrological_year.tif'

# variables
cloud_threshold = 10
NDSI_threshold_not_so_certain_snow = 10
NDSI_threshold_quite_certain_snow = 40

### Important Function:

#### 3-Day Temporal Interpolation:
Function to perform a 3-day temporal interpolation on the Terra files that were stacked to an array. The primary purpose is to fill in missing values within the array based on the values from the previous and and the next days.

#### SRTM Snowline Interpolation:
This function performs an interpolation on the array based on a Digital Elevation Model (DEM) and a cloud cover threshold. It iterates over each layer in the data stack and calculates the precentage of cloud pixels in the current scene. Then it checks whether this values is below the cloud threshold of 10%. If it is not, the scene is skiped. Otherwise, the function proceeds with the interpolation and idetifies three different types of pixels (free = pixel values between 32 and 64; snow = pixel values >= 64; fullsnow = pixels with values >= 128). After that, it calculates snowlines for different scenarios:

 - Snow Free Snowline: max. elevation with non-snow pixels
    
 - Snow Covered Snowline: min elevation with fully snow-covered pixels (if there are no pixels of this type, it uses the maximum elevation from the DEM)
    
After that, it updates the data of the current scene and assigns different values for each of the scenarios:
 - 66: pixel is originally cloud-covered and its elevation is above the snow free snowline (interpreted as a transition from cloud to snow)
 - 34: pixel is originally cloud-covered and its elevation is below the snow covered snowline (interpreted as a transition from cloud to free of snow)
 - 130: pixel is in a state of transition and its elevation is above the snow fullcovered snowline
 
Finally, it updates the stack for the current scene with the modified version.

#### Seasonal Interpolation:
The function initializes two arrays which are used to keep track of valid data and the number of days since the last cloud-free observation.Then it enters a loop, that iterates over each scene of the data stack. Inside the loop, it creates an output folder and constructs a string which represents the name of the output file. Then, it makes a copy of the input data stack and fills missing values by interpolation. Additionally, it tracks the number of days since the last cloud-free observation in an array (initially filled with zeros) by incrementing values by 1 where the data stack is not 0. Then it iterates over each scene and performs interpolation to handle missing values and set a seasonal bit under certain conditions. It keeps track of the number of days since the last cloud-free observation in a separate array. Lastly, it  performs a seasonal interpolation on the stack to estimate snow cover information for each pixel in each scene and stores the results as individual TIFF files. 

In [ ]:
# functions:

# read in a MODIS .hdf files
def read_MODIS_snow(file):
    hdf_ds = gdal.Open(file, gdal.GA_ReadOnly)
    band_ds = gdal.Open(hdf_ds.GetSubDatasets()[0][0], gdal.GA_ReadOnly) # 'Snow_Cover_Daily_Tile' v5, 'NDSI_Snow_Cover' v6
    data = band_ds.ReadAsArray()
    temp = np.copy(data)
    data = np.full(temp.shape, 0, dtype=np.uint8) # Missing Data (cloud, polar night etc.)
    data = np.where(temp < NDSI_threshold_not_so_certain_snow, 32, data) # Snow Free (NDSI < 0.1)
    data = np.where((temp >= NDSI_threshold_not_so_certain_snow) & (temp < NDSI_threshold_quite_certain_snow), 64, data) # Snow not so certain  (NDSI >= 0.1 & NDSI < 0.4)
    data = np.where((temp >= NDSI_threshold_quite_certain_snow) & (temp <= 100), 128, data) # Snow (NDSI >= 0.4)
    data = np.where(temp == 237, 16, data) # Inland Water
    data = np.where(temp == 239, 8, data) # Ocean
    snow_qa_ds = gdal.Open(hdf_ds.GetSubDatasets()[2][0], gdal.GA_ReadOnly) # "NDSI_Snow_Cover_Algorithm_Flags_QA"
    snow_qa = snow_qa_ds.ReadAsArray()
    snow_qa = snow_qa[:,:,np.newaxis]
    snow_qa_bits = np.unpackbits(snow_qa, axis=-1, bitorder='little') # Bit 0: Inland water
    data = np.where((data > 16) & (snow_qa_bits[:,:,0]==1), 16, data) #Pixels are re-assigned to water
    return data

# read in a geotiff file using GDAL
def read_tiff(file):
    ds = gdal.Open(file, gdal.GA_ReadOnly)
    band = ds.GetRasterBand(1)
    arr = band.ReadAsArray()
    return arr

# write geotiffs using GDAL (commented out the compresion because compressed files can not be read in later)
def write_tiff(outfile, data, proj_info, dtype=gdal.GDT_Byte):
    x,y = data.shape
    dst = gdal.GetDriverByName('GTiff').Create(outfile,y,x,1,dtype)#options=['COMPRESS=DEFLATE']
    dst.SetGeoTransform(proj_info[0])
    dst.SetProjection(proj_info[1])
    dst.GetRasterBand(1).WriteArray(data)
    dst = None
    return ''

# write 26 bit geotiffs using GDAL (commented out the compresion because compressed files can not be read in later)
def write_tiff(outfile, data, proj_info, dtype=gdal.GDT_UInt16):
    x,y = data.shape
    dst = gdal.GetDriverByName('GTiff').Create(outfile,y,x,1,dtype) #options=['COMPRESS=DEFLATE']
    dst.SetGeoTransform(proj_info[0])
    dst.SetProjection(proj_info[1])
    dst.GetRasterBand(1).WriteArray(data)
    dst = None
    return ''

# 3-day temporal interpolation
def interpolate_3day(GSP_data_stack, missing_value=0):
    temporary_GSP_stack = np.copy(GSP_data_stack)
    N = GSP_data_stack.shape[2]
    for j in range(N-1):
        if j>0:
            actual_data = np.copy(temporary_GSP_stack[:,:,j]) # today's data
            prior_data = np.copy(temporary_GSP_stack[:,:,j-1])
            next_data = np.copy(temporary_GSP_stack[:,:,j+1])
            actual_data = np.where(actual_data == missing_value, next_data, actual_data)
            actual_data = np.where(actual_data == missing_value, prior_data, actual_data)
            changed_values = np.where((temporary_GSP_stack[:,:,j] == missing_value) & (actual_data != missing_value))
            actual_data[changed_values]+=1
            GSP_data_stack[:,:,j] = actual_data[:,:]
    temporary_GSP_stack = None
    return GSP_data_stack

# SRTM snowline interpolation
def srtm_interpolation(GSP_data_stack, dem, cloud_threshold):
    N = GSP_data_stack.shape[2]
    for j in range(N):
        all_cloud_pixels=np.count_nonzero(GSP_data_stack[:,:,j] == 0)
        cloud_stats=all_cloud_pixels/GSP_data_stack[:,:,j].size * 100
        if cloud_stats<=cloud_threshold:
            #print("Entered if with {}".format(cloud_stats))
            print('Cloud cover below ' + str(cloud_threshold)+ ' in scene '+ str(j))
            free = np.where((GSP_data_stack[:,:,j] >= 32) & (GSP_data_stack[:,:,j] < 64))
            snow = np.where(GSP_data_stack[:,:,j] >= 64)
            fullsnow = np.where(GSP_data_stack[:,:,j] >= 128)
            step3_data_subset = np.copy(GSP_data_stack[:,:,j])
            if len(free[0])!=0:
                snow_free_snowline = np.nanmax(dem[free])
            else:
                snow_free_snowline = 0.0
            if len(snow[0])!=0:
                snow_covered_snowline = np.nanmin(dem[snow])
                if len(fullsnow[0])!=0:
                    snow_fullcovered_snowline = np.nanmax(dem[snow])
                else:
                    snow_fullcovered_snowline = np.nanmax(dem)
            else:
                snow_covered_snowline = np.nanmax(dem)
                snow_fullcovered_snowline = np.nanmax(dem)
            #find the pixels to recode:
            step3_data_subset = np.where((step3_data_subset == 0) & (dem > snow_free_snowline), 66, step3_data_subset) # cloud to snow
            step3_data_subset = np.where((step3_data_subset == 0) & (dem < snow_covered_snowline), 34, step3_data_subset) # cloud to free
            step3_data_subset = np.where((step3_data_subset == 66) & (dem > snow_fullcovered_snowline), 130, step3_data_subset)
            GSP_data_stack[:,:,j] = step3_data_subset[:,:] 
    return GSP_data_stack

# seasonal interpolation
def seasonal_interpolation(GSP_data_stack, proj_info):
    N = GSP_data_stack.shape[2]
    last_valid = np.full([2400,2400],1,dtype=np.uint8)
    days_to_cloudfree = np.full([2400,2400],0,dtype=np.uint16)
    for j in range(N):
        print(j)
        os.makedirs(SC_output_folder, exist_ok=True)
        season_name = 'SEASON10A1.A'+str(j+1)+'.tif'
        if j < 100:
            season_name = 'SEASON10A1.A0'+str(j+1)+'.tif'
        if j < 10:
            season_name = 'SEASON10A1.A00'+str(j+1)+'.tif'
        #accuracy_name = 'ACC10A1.A%04i%03i.%s.tif'%(timestamp.tm_year, timestamp.tm_yday, xdct['tile'])
        #cloud_distance_name = 'XCC10A1.A%04i%03i.%s.tif'%(timestamp.tm_year, timestamp.tm_yday, xdct['tile'])
        tmp_gsp = np.copy(GSP_data_stack[:,:,j])
        tmp_gsp = np.where((tmp_gsp==0)&(last_valid!=0),last_valid,tmp_gsp)
        seasonal_bit = np.unpackbits(tmp_gsp[:,:,np.newaxis], axis=-1, bitorder='little')[:,:,2] # check if seasonal bit is already set
        days_to_cloudfree = np.where((GSP_data_stack[:,:,j]!=0),0,days_to_cloudfree+1)
        last_valid = np.where((GSP_data_stack[:,:,j]!=0),GSP_data_stack[:,:,j],last_valid)
        tmp_gsp = np.where((days_to_cloudfree!=0)&(tmp_gsp>=8)&(seasonal_bit==0),tmp_gsp+4,tmp_gsp)
        #tmp_gsp = np.where((tmp_gsp != 0)&(land_mask == 0),0,tmp_gsp) # mask no data
        GSP_data_stack[:,:,j] = tmp_gsp
        write_tiff(os.path.join(SC_output_folder, season_name), tmp_gsp, proj_info)
    return ''

In [ ]:
# create two lists (MODIS Terra and Aqua) and fill them with the respective file names
# for this analysis we only need the terra_files

# create two empty lists
terra_files, aqua_files = [], []

for filename in os.listdir(MODIS_terra_file_folder)+os.listdir(MODIS_aqua_file_folder):
            #add_log_entry(log_file,'File found %s'%filename)
            version = filename.split('.')[3]
            if version == '006' or version == '061':
                if filename.startswith('MOD10A1.') & filename.endswith('hdf'):
                    terra_file = os.path.join(MODIS_terra_file_folder, filename)
                    #add_log_entry(log_file,'Terra File found %s'%terra_file)
                    terra_files.append(terra_file)
                if filename.startswith('MYD10A1.') & filename.endswith('hdf'):
                    aqua_file = os.path.join(MODIS_aqua_file_folder, filename)
                    #add_log_entry(log_file,'Aqua File found %s'%aqua_file)
                    aqua_files.append(aqua_file)

In [ ]:
terra_files

In [ ]:
# read in all the files from the terra_files list and store the information in an array (GSP_data_stack)
N = len(terra_files)
GSP_data_stack = np.full([2400, 2400, N], 0, dtype = np.uint8)

for i in range(len(terra_files)):
    print(terra_files[i])
    terra_data = read_MODIS_snow(terra_files[i])
    GSP_data_stack[:,:,i] = terra_data

In [ ]:
GSP_data_stack[:,:,150]

### Plot of the Cloud Cover Percentage
Visualization of the cloud coverage for each day as well as the min, max, and mean of one examplary year to get a better understanding of the data gaps due to clouds.

In [ ]:
# TASK: Plot the cloud cover percentage for each day as well as the min, max, and mean of one examplary year 

# get the cloud cover percentage for every scene of the year
N = GSP_data_stack.shape[2]
cloud_stats_list = []

for j in range(N):
    all_cloud_pixels = np.count_nonzero(GSP_data_stack[:,:,j] == 0)
    cloud_stats = all_cloud_pixels / GSP_data_stack[:,:,j].size*100
    cloud_stats_list.append(cloud_stats)

# create x-axis values corresponding to the day of the hydrological year
x_values = range(len(cloud_stats_list))

# plot the list with the cloud cover statistics as a line plot
plt.plot(x_values, cloud_stats_list, marker = 'o', markersize = 3, linestyle = '-')

# calculate the min, max, and mean percentage
mean_percentage = sum(cloud_stats_list) / len(cloud_stats_list)
min_value = min(cloud_stats_list)
max_value = max(cloud_stats_list)

# plot the min, max, and mean percentage as a horizontal line
plt.axhline(y = mean_percentage, color = 'r', linestyle = '--', label = f'Mean Percentage: {mean_percentage:.2f}')
plt.axhline(y = min_value, color = 'g', linestyle = '--', label = f'Minimum Value: {min_value:.2f}')
plt.axhline(y = max_value, color = 'b', linestyle = '--', label = f'Maximum Value: {max_value:.2f}')

# Add a horizontal line at y = 0.5
y_horizontal = 10
plt.axhline(y_horizontal, color='yellow', linestyle='--', label='Horizontal Line at y=0.5')


# add labels, title, and legend
plt.xlabel('Day of the Year')
plt.ylabel('Cloud Cover [%]')
plt.title('Cloud Cover')
plt.legend(loc = 'center left', bbox_to_anchor = (1.0, 0.8))

plt.show()

In [ ]:
# 3-day temporal interpolation
GSP_data_stack = interpolate_3day(GSP_data_stack)

In [ ]:
# srtm snowline elevation interpolation
dem = read_tiff(SRTM)
GSP_data_stack = srtm_interpolation(GSP_data_stack, dem, cloud_threshold)

In [ ]:
# seasonal interpolation

# get the projection information to be able to write out the result
hdf_ds = gdal.Open(terra_files[1])
ds = gdal.Open(hdf_ds.GetSubDatasets()[0][0])
proj_info = ds.GetGeoTransform(), ds.GetProjection()

# full seasonal filter
done = seasonal_interpolation(GSP_data_stack, proj_info)

In [ ]:
hdf_ds = gdal.Open(terra_files[1])
ds = gdal.Open(hdf_ds.GetSubDatasets()[0][0])
proj_info = ds.GetGeoTransform(), ds.GetProjection()

In [ ]:
# stack snow cover ouput rasters to an array

# list of SC output files
SC_output_files = []

for filename in os.listdir(SC_output_folder):
    if filename.endswith('tif'):
        SC_output_file = os.path.join(SC_output_folder, filename)
        SC_output_files.append(SC_output_file)

# open the SC raster files and stack them in an array
N = len(SC_output_files)
raster_data_list = []

for i in range(N):
    # open the .tif SC file
    with rasterio.open(SC_output_files[i]) as src:
        # read the raster data as  array
        raster_data_list.append(src.read(1))
        
raster_data_array = np.stack(raster_data_list, axis=0)

### Calculation of the SCD
A binary snow/ no snow mask is created, where all pixels that are less than 128 (no snow) are set to 0. Similarly, all pixel values greater than or equal 128 are set to 1 (snow). Then, the SCD is calculated for each pixel by summing up the binary values (0 or 1) for every scene of the hydrological year. The result is an array, where each element represets the SCD for the corresponding pixel.

In [ ]:
# calculate the SCD

# copy the array (to keep the unchanged original)
rda = np.copy(raster_data_array)

# binary snow/ no snow mask
# set all snow pixels (pixels > 128) to 1 and all no snow pixels (pixels < 128) to 0
rda[rda < 128] = 0
rda[rda >= 128] = 1

# count the number of days with snow cover (1) per pixel
rda_sum = np.sum(rda, axis = 0)

# plot showing the snow cover duration for the hydrological year
plt.imshow(rda_sum)
plt.title('Snow Cover Duration one Hydrological Year')
plt.colorbar()

plt.show()

In [ ]:
# save the array containing the SCD
np.save(os.path.join(SCD_output_folder, SCD_output_array), rda_sum)

In [ ]:
# save the SCD array as .tif file
write_tiff(os.path.join(SCD_output_folder, SCD_output_raster), rda_sum, proj_info)

In [ ]:
rda_sum.dtype